In [52]:
%cd thesis
import sys

sys.path.append(os.getcwd())

[Errno 2] No such file or directory: 'thesis'
/home/jupyter/work/resources/thesis


In [25]:
%pip install -r requirements.conda.txt

Defaulting to user installation because normal site-packages is not writeable
  Using cached realesrgan-0.2.3.0-py3-none-any.whl (25 kB)
  Using cached torchsummary-1.5.1-py3-none-any.whl (2.8 kB)
  Using cached basicsr-1.3.4.9.tar.gz (161 kB)
  Preparing metadata (setup.py) ... done
  Using cached facexlib-0.2.1.1-py3-none-any.whl (56 kB)
  Using cached gfpgan-0.2.4-py3-none-any.whl (38 kB)
  Using cached opencv_python-4.5.5.62-cp36-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (60.4 MB)
  Using cached addict-2.4.0-py3-none-any.whl (3.8 kB)
  Using cached lmdb-1.3.0-cp38-cp38-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (305 kB)
  Using cached tb_nightly-2.9.0a20220201-py3-none-any.whl (5.8 MB)
  Using cached yapf-0.32.0-py2.py3-none-any.whl (190 kB)
  Using cached filterpy-1.4.5.zip (177 kB)
  Preparing metadata (setup.py) ... done
  Created wheel for basicsr: filename=basicsr-1.3.4.9-py3-none-any.whl size=194439 sha256=648ff3415a83de837d27cc3cbf74a5bb0b47123b954ef79644279a35

In [26]:
import torch
import os
from torchsummary import summary
from realesrgan import RealESRGANer
from basicsr.archs.rrdbnet_arch import RRDBNet

os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"


In [48]:
torch.cuda.is_available()

False

In [55]:
# Upsampler

model = RRDBNet(num_in_ch=3, num_out_ch=3, num_feat=64)

upsampler = RealESRGANer(
    scale=4,
    model_path='./realesrgan/experiments/pretrained_models/RealESRGAN_x4plus.pth',
    model=model,
    tile=False,
    tile_pad=10,
    pre_pad=0,
    half=False)

FileNotFoundError: [Errno 2] No such file or directory: './realesrgan/experiments/pretrained_models/RealESRGAN_x4plus.pth'

In [50]:
from src.helpers import utils
from compress import make_deterministic
from src.loss.perceptual_similarity import perceptual_loss as ps
from default_config import ModelModes

In [57]:
# Compression

# Reproducibility
make_deterministic()
perceptual_loss_fn = ps.PerceptualLoss(model='net-lin', net='alex', use_gpu=torch.cuda.is_available())

# Load model
device = torch.device('cpu')#utils.get_device()
print(device)
logger = utils.logger_setup(logpath=os.path.join('images', 'logs'), filepath=os.path.abspath('1'))
loaded_args, compression, _ = utils.load_model('experiments/hific_low.pt', logger, device, model_mode=ModelModes.EVALUATION,
    current_args_d=None, prediction=True, strict=False)
print('logger done')

#compression.Hyperprior.hyperprior_entropy_model.build_tables()

17:32:56 INFO - logger_setup: /home/jupyter/work/resources/thesis/1
17:32:56 INFO - logger_setup: /home/jupyter/work/resources/thesis/1
17:32:56 INFO - logger_setup: /home/jupyter/work/resources/thesis/1
17:32:56 INFO - logger_setup: /home/jupyter/work/resources/thesis/1
17:32:56 INFO - logger_setup: /home/jupyter/work/resources/thesis/1
100%|██████████| 64/64 [00:00<00:00, 356.12it/s]
17:35:00 INFO - load_model: Loading model ...
17:35:00 INFO - load_model: Loading model ...
17:35:00 INFO - load_model: Loading model ...
17:35:00 INFO - load_model: Loading model ...
17:35:00 INFO - load_model: Loading model ...
17:35:00 INFO - load_model: MODEL TYPE: compression_gan
17:35:00 INFO - load_model: MODEL TYPE: compression_gan
17:35:00 INFO - load_model: MODEL TYPE: compression_gan
17:35:00 INFO - load_model: MODEL TYPE: compression_gan
17:35:00 INFO - load_model: MODEL TYPE: compression_gan
17:35:00 INFO - load_model: MODEL MODE: evaluation
17:35:00 INFO - load_model: MODEL MODE: evaluation

Setting up Perceptual loss...
Loading model from: /home/jupyter/work/resources/thesis/src/loss/perceptual_similarity/weights/v0.1/alex.pth
...[net-lin [alex]] initialized
...Done
cpu
Building prior probability tables...
Setting up Perceptual loss...
Loading model from: /home/jupyter/work/resources/thesis/src/loss/perceptual_similarity/weights/v0.1/alex.pth
...[net-lin [alex]] initialized
...Done
logger done


In [59]:
# Input size: [2, 220, 8, 8]

input = torch.rand(1, 220, 8, 8)

result = compression.Generator(input)
summary(compression.Generator, (220, 8, 8), 1, device='cpu')


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
     ChannelNorm2D-1             [1, 220, 8, 8]               0
   ReflectionPad2d-2           [1, 220, 10, 10]               0
   ReflectionPad2d-3           [1, 220, 10, 10]               0
            Conv2d-4             [1, 960, 8, 8]       1,901,760
     ChannelNorm2D-5             [1, 960, 8, 8]               0
   ReflectionPad2d-6           [1, 960, 10, 10]               0
            Conv2d-7             [1, 960, 8, 8]       8,295,360
     ChannelNorm2D-8             [1, 960, 8, 8]               0
   ReflectionPad2d-9           [1, 960, 10, 10]               0
           Conv2d-10             [1, 960, 8, 8]       8,295,360
    ChannelNorm2D-11             [1, 960, 8, 8]               0
    ResidualBlock-12             [1, 960, 8, 8]               0
  ReflectionPad2d-13           [1, 960, 10, 10]               0
           Conv2d-14             [1, 96

In [60]:
summary(compression, (3, 128, 128), 2, device='cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
   ReflectionPad2d-1           [2, 3, 134, 134]               0
   ReflectionPad2d-2           [2, 3, 134, 134]               0
            Conv2d-3          [2, 60, 128, 128]           8,880
     ChannelNorm2D-4          [2, 60, 128, 128]               0
              ReLU-5          [2, 60, 128, 128]               0
   ReflectionPad2d-6          [2, 60, 129, 129]               0
   ReflectionPad2d-7          [2, 60, 129, 129]               0
   ReflectionPad2d-8          [2, 60, 129, 129]               0
   ReflectionPad2d-9          [2, 60, 129, 129]               0
  ReflectionPad2d-10          [2, 60, 129, 129]               0
           Conv2d-11           [2, 120, 64, 64]          64,920
    ChannelNorm2D-12           [2, 120, 64, 64]               0
             ReLU-13           [2, 120, 64, 64]               0
  ReflectionPad2d-14           [2, 120,

/kernel/lib/python3.8/site-packages/numpy/core/fromnumeric.py:87: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


TypeError: can't multiply sequence by non-int of type 'list'

In [61]:
summary(upsampler.model, (3, 128, 128), 1, device='cpu')

NameError: name 'upsampler' is not defined

In [62]:
compression.train(False)
compression(torch.rand((16, 3, 128, 128)))

(tensor([[[[0.7440, 0.7595, 0.7649,  ..., 0.7502, 0.7729, 0.7499],
           [0.7460, 0.7439, 0.7479,  ..., 0.7267, 0.7630, 0.7519],
           [0.7240, 0.7518, 0.7720,  ..., 0.7460, 0.7700, 0.7553],
           ...,
           [0.7602, 0.7695, 0.7524,  ..., 0.7332, 0.7431, 0.7577],
           [0.7697, 0.7790, 0.7673,  ..., 0.7373, 0.7333, 0.7527],
           [0.7766, 0.7952, 0.7854,  ..., 0.7635, 0.7356, 0.7380]],
 
          [[0.7531, 0.7618, 0.7628,  ..., 0.7473, 0.7645, 0.7329],
           [0.7456, 0.7349, 0.7376,  ..., 0.7210, 0.7521, 0.7351],
           [0.7248, 0.7477, 0.7639,  ..., 0.7524, 0.7655, 0.7395],
           ...,
           [0.7719, 0.7779, 0.7592,  ..., 0.7340, 0.7433, 0.7556],
           [0.7761, 0.7823, 0.7676,  ..., 0.7410, 0.7357, 0.7511],
           [0.7770, 0.7948, 0.7889,  ..., 0.7655, 0.7365, 0.7308]],
 
          [[0.7570, 0.7779, 0.7804,  ..., 0.7727, 0.7882, 0.7655],
           [0.7579, 0.7613, 0.7643,  ..., 0.7439, 0.7765, 0.7695],
           [0.7298, 0.76

In [63]:
class Model(torch.nn.Module):
    def __init__(self, compression, upsampler):
        super().__init__()
        self.compression = compression
        self.upsampler = upsampler
    
    def forward(self, x):
        y, loss = self.compression(x)
        y = self.upsampler(y)
        return y

    def train(self, train=True):
        self.compression.train(False)
        self.compression.Generator.train(train)
        self.upsampler.train(train)

In [64]:
m = Model(compression, upsampler.model)

NameError: name 'upsampler' is not defined

In [ ]:
x = torch.rand(1, 3, 128, 128)
result = m(x)

print(result.shape)

### Remarks
- Conv2D with 2x2 padding that's equivalent to Conv2DTranspose with no padding.
- Checkerboard artifacts can start to become an issue when using strides (even after stacking multiple layers).

### Things to try: 

1. To avoid checkerboard artifacts, an alternative upsampling method that’s gaining popularity is to apply classical upsampling followed by a regular convolution (that preserves the spatial dimensions).

In [67]:
import torch
import torch.nn.functional as F
from src.normalisation import channel, instance

class ResidualBlock(torch.nn.Module):
    def __init__(self, input_dims, kernel_size=3, stride=1, 
                 channel_norm=True, activation='relu'):
        """
        input_dims: Dimension of input tensor (B,C,H,W)
        """
        super(ResidualBlock, self).__init__()

        self.activation = getattr(F, activation)
        in_channels = input_dims[1]
        norm_kwargs = dict(momentum=0.1, affine=True, track_running_stats=False)

        if channel_norm is True:
            self.interlayer_norm = channel.ChannelNorm2D_wrap
        else:
            self.interlayer_norm = instance.InstanceNorm2D_wrap

        pad_size = int((kernel_size-1)/2)
        self.pad = torch.nn.ReflectionPad2d(pad_size)
        self.conv1 = torch.nn.Conv2d(in_channels, in_channels, kernel_size, stride=stride)
        self.conv2 = torch.nn.Conv2d(in_channels, in_channels, kernel_size, stride=stride)
        self.norm1 = self.interlayer_norm(in_channels, **norm_kwargs)
        self.norm2 = self.interlayer_norm(in_channels, **norm_kwargs)

    def forward(self, x):
        identity_map = x
        res = self.pad(x)
        res = self.conv1(res)
        res = self.norm1(res) 
        res = self.activation(res)

        res = self.pad(res)
        res = self.conv2(res)
        res = self.norm2(res)

        return torch.add(res, identity_map)

class Upsampler(torch.nn.Module):

    def __init__(self, input_dims, batch_size, C=220, activation='relu',
                 n_residual_blocks=8, channel_norm=True, sample_noise=False,
                 noise_dim=32, silent=True):
        super(Upsampler, self).__init__()
        self.silent = silent

        kernel_dim = 3
        filters = [960, 480, 240, 120, 60]
        self.n_residual_blocks = n_residual_blocks
        self.sample_noise = sample_noise
        self.noise_dim = noise_dim

        # Layer / normalization options
        cnn_kwargs = dict(stride=2, padding=1, output_padding=1)
        norm_kwargs = dict(momentum=0.1, affine=True, track_running_stats=False)
        activation_d = dict(relu='ReLU', elu='ELU', leaky_relu='LeakyReLU')
        self.activation = getattr(torch.nn, activation_d[activation])  # (leaky_relu, relu, elu)
        self.n_upsampling_layers = 4
        
        if channel_norm is True:
            self.interlayer_norm = channel.ChannelNorm2D_wrap
        else:
            self.interlayer_norm = instance.InstanceNorm2D_wrap

        self.pre_pad = torch.nn.ReflectionPad2d(1)
        self.asymmetric_pad = torch.nn.ReflectionPad2d((0,1,1,0))  # Slower than tensorflow?
        self.post_pad = torch.nn.ReflectionPad2d(3)

        H0, W0 = input_dims[1:]
        heights = [2**i for i in range(5,9)]
        widths = heights
        H1, H2, H3, H4 = heights
        W1, W2, W3, W4 = widths 


        # (16,16) -> (16,16), with implicit padding
        self.conv_block_init = torch.nn.Sequential(
            self.interlayer_norm(C, **norm_kwargs),
            self.pre_pad,
            torch.nn.Conv2d(C, filters[0], kernel_size=(3,3), stride=1),
            self.interlayer_norm(filters[0], **norm_kwargs),
        )

        if sample_noise is True:
            # Concat noise with latent representation
            filters[0] += self.noise_dim

        for m in range(n_residual_blocks):
            resblock_m = ResidualBlock(input_dims=(batch_size, filters[0], H0, W0), 
                channel_norm=channel_norm, activation=activation)
            self.add_module(f'resblock_{str(m)}', resblock_m)
        
        self.upconv_block1 = torch.nn.Sequential(
            torch.nn.ConvTranspose2d(filters[0], filters[1], kernel_dim, **cnn_kwargs),
            self.interlayer_norm(filters[1], **norm_kwargs),
            self.activation(),
        )
        self.upconv_block2 = torch.nn.Sequential(
            torch.nn.ConvTranspose2d(filters[1], filters[2], kernel_dim, **cnn_kwargs),
            self.interlayer_norm(filters[2], **norm_kwargs),
            self.activation(),
        )
        self.upconv_block3 = torch.nn.Sequential(
            torch.nn.ConvTranspose2d(filters[2], filters[3], kernel_dim, **cnn_kwargs),
            self.interlayer_norm(filters[3], **norm_kwargs),
            self.activation(),
        )
        self.upconv_block4 = torch.nn.Sequential(
            torch.nn.ConvTranspose2d(filters[3], filters[4], kernel_dim, **cnn_kwargs),
            self.interlayer_norm(filters[4], **norm_kwargs),
            self.activation(),
        )
        self.conv_block_out = torch.nn.Sequential(
            self.post_pad,
            torch.nn.Conv2d(filters[-1], 3, kernel_size=(7,7), stride=1),
        )

    def forward(self, x):
        if not self.silent:
            print(f'{"INPUT : ": <15}', x.shape)
        head = self.conv_block_init(x)
        if not self.silent:
            print(f'{"INITIAL_CONV : ": <15}', head.shape)

        if self.sample_noise is True:
            B, C, H, W = tuple(head.size())
            z = torch.randn((B, self.noise_dim, H, W)).to(head)
            head = torch.cat((head,z), dim=1)

        for m in range(self.n_residual_blocks):
            resblock_m = getattr(self, f'resblock_{str(m)}')
            if m == 0:
                x = resblock_m(head)
            else:
                x = resblock_m(x)
            if not self.silent:
                print(f'{f"RESIDUAL_{m}": <15}', x.shape)
        
        x += head
        x = self.upconv_block1(x)
        if not self.silent:
            print(f'{"UPCONV_1 : ": <15}', x.shape)
        x = self.upconv_block2(x)
        if not self.silent:
            print(f'{"UPCONV_2 : ": <15}', x.shape)
        x = self.upconv_block3(x)
        if not self.silent:
            print(f'{"UPCONV_3 : ": <15}', x.shape)
        x = torch.nn.functional.interpolate(x, scale_factor=2, mode='nearest')
        if not self.silent:
            print(f'{"BILINEAR : ": <15}', x.shape)
        x = self.upconv_block4(x)
        if not self.silent:
            print(f'{"UPCONV_4 : ": <15}', x.shape)
        out = self.conv_block_out(x)
        if not self.silent:
            print(f'{"UPCONV_5 : ": <15}', out.shape)

        return out

In [ ]:
upsampler = Upsampler([220, 8, 8], 2)

input = torch.rand(2, 220, 8, 8)
output = upsampler(input)

print(output.shape)

In [ ]:
summary(upsampler, (220, 8, 8), 2, device='cpu')

Try using interpolation to upscale feature maps. Need to match shapes.

In [ ]:
import torch
from torch.nn import functional as F

shape = (2, 220, 8, 8)

input = torch.rand(shape)
output1 = F.interpolate(input, scale_factor=2, mode='nearest')

print(input.shape, output1.shape)

### Load model from checkpoint and modify structure of Generator

In [65]:
# Compression
import torch
from src.helpers import utils
from compress import make_deterministic
from src.loss.perceptual_similarity import perceptual_loss as ps
from default_config import ModelModes

# Reproducibility
make_deterministic()
perceptual_loss_fn = ps.PerceptualLoss(model='net-lin', net='alex', use_gpu=torch.cuda.is_available())

# Load model
device = torch.device('cpu')#utils.get_device()
logger = utils.logger_setup(logpath=os.path.join('images', 'logs'), filepath=os.path.abspath('1'))
loaded_args, compression, _ = utils.load_model('experiments/hific_low.pt', logger, device, model_mode=ModelModes.EVALUATION,
    current_args_d=None, prediction=True, strict=False, silent=True)

#compression.Hyperprior.hyperprior_entropy_model.build_tables()

17:36:52 INFO - logger_setup: /home/jupyter/work/resources/thesis/1
17:36:52 INFO - logger_setup: /home/jupyter/work/resources/thesis/1
17:36:52 INFO - logger_setup: /home/jupyter/work/resources/thesis/1
17:36:52 INFO - logger_setup: /home/jupyter/work/resources/thesis/1
17:36:52 INFO - logger_setup: /home/jupyter/work/resources/thesis/1
17:36:52 INFO - logger_setup: /home/jupyter/work/resources/thesis/1
100%|██████████| 64/64 [00:00<00:00, 458.19it/s]
17:36:55 INFO - load_model: Loading model ...
17:36:55 INFO - load_model: Loading model ...
17:36:55 INFO - load_model: Loading model ...
17:36:55 INFO - load_model: Loading model ...
17:36:55 INFO - load_model: Loading model ...
17:36:55 INFO - load_model: Loading model ...
17:36:55 INFO - load_model: Estimated model size (under fp32): 593.146 MB
17:36:55 INFO - load_model: Estimated model size (under fp32): 593.146 MB
17:36:55 INFO - load_model: Estimated model size (under fp32): 593.146 MB
17:36:55 INFO - load_model: Estimated model s

Setting up Perceptual loss...
Loading model from: /home/jupyter/work/resources/thesis/src/loss/perceptual_similarity/weights/v0.1/alex.pth
...[net-lin [alex]] initialized
...Done
Building prior probability tables...
Setting up Perceptual loss...
Loading model from: /home/jupyter/work/resources/thesis/src/loss/perceptual_similarity/weights/v0.1/alex.pth
...[net-lin [alex]] initialized
...Done


In [68]:
upsampler = Upsampler((220, 8, 8), 1)

In [69]:
compression.train(False)
compression.Generator = upsampler
compression.Generator.train(True)

Upsampler(
  (pre_pad): ReflectionPad2d((1, 1, 1, 1))
  (asymmetric_pad): ReflectionPad2d((0, 1, 1, 0))
  (post_pad): ReflectionPad2d((3, 3, 3, 3))
  (conv_block_init): Sequential(
    (0): ChannelNorm2D()
    (1): ReflectionPad2d((1, 1, 1, 1))
    (2): Conv2d(220, 960, kernel_size=(3, 3), stride=(1, 1))
    (3): ChannelNorm2D()
  )
  (resblock_0): ResidualBlock(
    (pad): ReflectionPad2d((1, 1, 1, 1))
    (conv1): Conv2d(960, 960, kernel_size=(3, 3), stride=(1, 1))
    (conv2): Conv2d(960, 960, kernel_size=(3, 3), stride=(1, 1))
    (norm1): ChannelNorm2D()
    (norm2): ChannelNorm2D()
  )
  (resblock_1): ResidualBlock(
    (pad): ReflectionPad2d((1, 1, 1, 1))
    (conv1): Conv2d(960, 960, kernel_size=(3, 3), stride=(1, 1))
    (conv2): Conv2d(960, 960, kernel_size=(3, 3), stride=(1, 1))
    (norm1): ChannelNorm2D()
    (norm2): ChannelNorm2D()
  )
  (resblock_2): ResidualBlock(
    (pad): ReflectionPad2d((1, 1, 1, 1))
    (conv1): Conv2d(960, 960, kernel_size=(3, 3), stride=(1, 1))


In [70]:
input = torch.rand((1, 3, 128, 128))
output = compression(input)

print(f'OUTPUT SHAPE : {output[0].shape}')

OUTPUT SHAPE : torch.Size([1, 3, 128, 128])


### Now we got to train the network

For this purpose we need:

1. Set up a dataset
2. Run training script from hific repo

In [104]:
%pip install awscli

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.


In [108]:
%pip install boto3

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 6.2 MB 1.2 MB/s            
     |████████████████████████████████| 73 kB 1.5 MB/s             
  Attempting uninstall: botocore
    Found existing installation: botocore 1.23.47
    Uninstalling botocore-1.23.47:
      Successfully uninstalled botocore-1.23.47
  Attempting uninstall: s3transfer
    Found existing installation: s3transfer 0.5.1
    Uninstalling s3transfer-0.5.1:
      Successfully uninstalled s3transfer-0.5.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
awscli 1.22.47 requires botocore==1.23.47, but you have botocore 1.15.49 which is incompatible.
awscli 1.22.47 requires s3transfer<0.6.0,>=0.5.0, but you have s3transfer 0.3.7 which is incompatible.
moto 1.3.14 requires idna<2.9,>=2.5, but you have idna 2.10 which is incompati

In [135]:
#pragma dataset init OPEN_IMAGES --size 128Gb

# TODO: fill dataset here
import boto3
from botocore.config import Config
from botocore import UNSIGNED

import tqdm

BUCKET = 'open-images-dataset'
KEY = 'tar/train_0.tar.gz'

s3 = boto3.client('s3', config=Config(signature_version=UNSIGNED))
metadata = s3.head_object(Bucket=BUCKET, Key=KEY)
print(f'Downloading file ({metadata["ResponseMetadata"]})...')
#with tqdm.tqdm()
#    s3.download_file(BUCKET, KEY, f'{DATASET_PATH}/train_0.tar.gz', callback=lambda bytes_transfered: )
print('Download completed.')
# Dataset will be created in /home/jupyter/mnt/datasets/OPEN_IMAGES

It's init dataset task. State result won't be merged.


SyntaxError: invalid syntax (<ipython-input-1-2a73e0e20a27>, line 16)

In [130]:
DATASET_PATH = '/home/jupyter/mnt/datasets/OPEN_IMAGES'
HOME_PATH = '/home/jupyter/work/resources/thesis'